In [ ]:
%load_ext line_profiler

In [ ]:
pip install solidity_parser

In [ ]:
import os
import json
import solcx
import solcast
import re
import pandas as pd
from solidity_parser import parser

In [ ]:
sourceUnit = parser.parse_file(f'./{dataset_name}/QuillAudits-1inch-token/1inch-token-99fd056f91005ca521a02a005f7bcd8f77e06afc/contracts/ERC20Permit.sol', loc=False) # loc=True -> add location information to ast nodes

In [ ]:
type(sourceUnit)
#type(sourceUnit.children)
# sourceUnit.children(
#     include_children=False,
#     filters={'nodeType': "FunctionCall", "expression.name": "require"}
# )

In [ ]:
#root_node = solcast.from_ast(sourceUnit)
#result = sourceUnit.children(required_offset=(1, 10))
#open('result.json', 'w').write(sourceUnit).close()
#sourceUnit = parser.parse_file(f'./sample.sol', loc=True) # loc=True -> add location information to ast nodes
#print(sourceUnit)
#sourceUnitObject = parser.objectify(sourceUnit)
#sourceUnitObject.contracts['EtherWallet'].functions['getBalance']
#sourceUnitObject.contracts['EtherWallet'].functions['getBalance']

In [ ]:
def get_function_line_numbers(ast, function_name):
    # Define a recursive function to search through the AST
    def search_ast(node):
        # Check if the node is a dictionary and has the key 'type'
        if isinstance(node, dict) and 'type' in node:
            # If the node is a FunctionDefinition and matches the function_name, return its line numbers
            if node['type'] == 'FunctionDefinition' and node.get('name') == function_name:
                start_line = node['loc']['start']['line']
                end_line = node['loc']['end']['line']
                return start_line, end_line
            
            # Iterate over all values in the node, which might be lists or dictionaries, to find nested nodes
            for value in node.values():
                result = search_ast(value)
                if result:  # If the function is found, return its line numbers
                    return result
        
        # If the node is a list, iterate through its items
        elif isinstance(node, list):
            for item in node:
                result = search_ast(item)
                if result:  # If the function is found, return its line numbers
                    return result

        return None  # If the function is not found, return None

    # Start the search from the root of the AST
    return search_ast(ast)

print(get_function_line_numbers(sourceUnit, 'getBalance'))

In [ ]:
sourceUnit = parser.parse_file(f'./sample.sol', loc=True) # loc=True -> add location information to ast nodes
sourceUnit
# def convert_ast_to_dict(node):
#     if isinstance(node, dict):
#         return {k: convert_ast_to_dict(v) for k, v in node.items()}
#     elif isinstance(node, list):
#         return [convert_ast_to_dict(elem) for elem in node]
#     elif hasattr(node, '__dict__'):
#         return {k: convert_ast_to_dict(v) for k, v in node.__dict__.items()}
#     else:
#         return node

# Assuming `sourceUnit` is the AST object returned by `solidity_parser`
# sourceUnit_dict = convert_ast_to_dict(sourceUnit)

# Now `sourceUnit_dict` should be a dictionary representation of the AST
# that you can use with `solcast.from_ast`
solcast_ast = solcast.from_ast(parser.objectify(sourceUnit))
#result = solcast_ast.children(
#        required_offset=(1, 4))
solcast_ast
#type(sourceUnit)

In [ ]:
def find_nodes_with_offset(node, required_offset):
    results = []
    if isinstance(node, dict):
        if "src" in node and node["src"] == required_offset:
            results.append(node)
        for key, value in node.items():
            results.extend(find_nodes_with_offset(value, required_offset))
    elif isinstance(node, list):
        for item in node:
            results.extend(find_nodes_with_offset(item, required_offset))
    return results

# Use this function to search through `sourceUnit_dict`
nodes_with_required_offset = find_nodes_with_offset(sourceUnit_dict, (1, 4))
print(nodes_with_required_offset)

In [ ]:

MAPPING = {
    'reentrancy': {
        "slither": [
            "reentrancy-benign",
            "reentrancy-events",
            "reentrancy-no-eth",
            "reentrancy-unlimited-gas",
            "reentrancy-eth",
        ],
        'semgrep': [
            "compound-borrowfresh-reentrancy",
            "erc721-reentrancy",
            "curve-readonly-reentrancy",
            "erc777-reentrancy",
            "erc677-reentrancy"
        ],
        'smartcheck': [

        ]
    },
    'low-level-calls': {
        'slither': [],
        'semgrep': [
            'arbitrary-low-level-call'
        ],
        'smartcheck': [
            'SOLIDITY_CALL_WITHOUT_DATA'
        ]
    },
    'access-control': {
        'slither': [
            "arbitrary-send-erc20",
            "arbitrary-send-eth",
            "suicidal",
            "arbitrary-send-erc20-permit",
        ],
        'semgrep': [
            'compound-sweeptoken-not-restricted',
            'erc20-public-burn',
            'accessible-selfdestruct',
            'oracle-price-update-not-restricted',
            'uniswap-callback-not-protected',

        ],
        'smartcheck': [
            'SOLIDITY_TX_ORIGIN'
        ]
    },
    'delegation': {
        'slither': [
            "controlled-delegatecall",
            "delegatecall-loop",
        ],
        'semgrep': [
            'delegatecall-to-arbitrary-address',

        ],
        'smartcheck': [

        ]
    },
    'arithmetic': {
        'slither': [
            "divide-before-multiply",
        ],
        'semgrep': [
            'basic-arithmetic-underflow'
        ],
        'smartcheck': [
            'SOLIDITY_DIV_MUL'
        ]
    },
    'oracle-manipulation': {
        'slither': [],
        'semgrep': [
            'keeper-network-oracle-manipulation'
        ],
        'smartcheck': [

        ]
    },
    'input-validation': {
        'slither': [
            "missing-zero-check",
        ],
        'semgrep': [

        ],
        'smartcheck': [

        ]
    },
    'shadowing': {
        'slither': [
            "shadowing-local",
            "shadowing-state",
            "shadowing-abstract",
        ],
        'semgrep': [

        ],
        'smartcheck': [

        ]
    },
    'compliance': {
        'slither': [
            "erc20-interface",
        ],
        'semgrep': [

        ],
        'smartcheck': [
            'SOLIDITY_ERC20_TRANSFER_SHOULD_THROW'
        ]
    },
    'timestamp': {
        "slither": [
            "timestamp",
            "weak-prng",
        ],
        'semgrep': [

        ],
        'smartcheck': [
        ]
    },
    'initialization': {
        'slither': [
            "uninitialized-local",
            "uninitialized-state",
        ],
        'semgrep': [

        ],
        'smartcheck': [

        ]
    },
    'poor-logic-flaws': {
        "slither": [
            "incorrect-equality",
            "boolean-cst",
        ],
        'semgrep': [
            'incorrect-use-of-blockhash'
        ],
        'smartcheck': [
            'SOLIDITY_EXACT_TIME',
            'SOLIDITY_BALANCE_EQUALITY'
        ]
    },
    'denial-of-service': {
        "slither": [
            "locked-ether",
            "calls-loop",
            "msg-value-loop",
        ],
        'semgrep': [

        ],
        'smartcheck': [
            'SOLIDITY_LOCKED_MONEY',
            'SOLIDITY_TRANSFER_IN_LOOP'
        ]
    },
    'state-corruption': {
        "slither": [
            "controlled-array-length",
        ],
        'semgrep': [

        ],
        'smartcheck': [
            'SOLIDITY_ARRAY_LENGTH_MANIPULATION'
        ]
    },
    'function-behavior': {
        "slither": [
            'incorrect-modifier'
        ],
        'semgrep': [

        ],
        'smartcheck': [

        ]
    },
    'external-call-validation': {
        "slither": [
            "unchecked-transfer",
            "unchecked-lowlevel"
        ],
        'semgrep': [

        ],
        'smartcheck': [

        ]
    },
    'front-running': {
        "slither": [],
        'semgrep': [

        ],
        'smartcheck': [
            'SOLIDITY_ERC20_APPROVE'
        ]
    }
}

REVERSE_MAPPING = {
    'reentrancy-benign': {'category': 'reentrancy', 'analyzer': 'slither'},
    'reentrancy-events': {'category': 'reentrancy', 'analyzer': 'slither'},
    'reentrancy-no-eth': {'category': 'reentrancy', 'analyzer': 'slither'},
    'reentrancy-unlimited-gas': {'category': 'reentrancy', 'analyzer': 'slither'},
    'reentrancy-eth': {'category': 'reentrancy', 'analyzer': 'slither'},
    'compound-borrowfresh-reentrancy': {'category': 'reentrancy', 'analyzer': 'semgrep'},
    'erc721-reentrancy': {'category': 'reentrancy', 'analyzer': 'semgrep'},
    'curve-readonly-reentrancy': {'category': 'reentrancy', 'analyzer': 'semgrep'},
    'erc777-reentrancy': {'category': 'reentrancy', 'analyzer': 'semgrep'},
    'erc677-reentrancy': {'category': 'reentrancy', 'analyzer': 'semgrep'},
    'arbitrary-low-level-call': {'category': 'low-level-calls', 'analyzer': 'semgrep'},
    'SOLIDITY_CALL_WITHOUT_DATA': {'category': 'low-level-calls', 'analyzer': 'smartcheck'},
    'arbitrary-send-erc20': {'category': 'access-control', 'analyzer': 'slither'},
    'arbitrary-send-eth': {'category': 'access-control', 'analyzer': 'slither'},
    'suicidal': {'category': 'access-control', 'analyzer': 'slither'},
    'arbitrary-send-erc20-permit': {'category': 'access-control', 'analyzer': 'slither'},
    'compound-sweeptoken-not-restricted': {'category': 'access-control', 'analyzer': 'semgrep'},
    'erc20-public-burn': {'category': 'access-control', 'analyzer': 'semgrep'},
    'accessible-selfdestruct': {'category': 'access-control', 'analyzer': 'semgrep'},
    'oracle-price-update-not-restricted': {'category': 'access-control', 'analyzer': 'semgrep'},
    'uniswap-callback-not-protected': {'category': 'access-control', 'analyzer': 'semgrep'},
    'SOLIDITY_TX_ORIGIN': {'category': 'access-control', 'analyzer': 'smartcheck'},
    'controlled-delegatecall': {'category': 'delegation', 'analyzer': 'slither'},
    'delegatecall-loop': {'category': 'delegation', 'analyzer': 'slither'},
    'delegatecall-to-arbitrary-address': {'category': 'delegation', 'analyzer': 'semgrep'},
    'divide-before-multiply': {'category': 'arithmetic', 'analyzer': 'slither'},
    'basic-arithmetic-underflow': {'category': 'arithmetic', 'analyzer': 'semgrep'},
    'SOLIDITY_DIV_MUL': {'category': 'arithmetic', 'analyzer': 'smartcheck'},
    'keeper-network-oracle-manipulation': {'category': 'oracle-manipulation', 'analyzer': 'semgrep'},
    'missing-zero-check': {'category': 'input-validation', 'analyzer': 'slither'},
    'shadowing-local': {'category': 'shadowing', 'analyzer': 'slither'},
    'shadowing-state': {'category': 'shadowing', 'analyzer': 'slither'},
    'shadowing-abstract': {'category': 'shadowing', 'analyzer': 'slither'},
    'erc20-interface': {'category': 'compliance', 'analyzer': 'slither'},
    'SOLIDITY_ERC20_TRANSFER_SHOULD_THROW': {'category': 'compliance', 'analyzer': 'smartcheck'},
    'timestamp': {'category': 'timestamp', 'analyzer': 'slither'},
    'weak-prng': {'category': 'timestamp', 'analyzer': 'slither'},
    'uninitialized-local': {'category': 'initialization', 'analyzer': 'slither'},
    'uninitialized-state': {'category': 'initialization', 'analyzer': 'slither'},
    'incorrect-equality': {'category': 'poor-logic-flaws', 'analyzer': 'slither'},
    'boolean-cst': {'category': 'poor-logic-flaws', 'analyzer': 'slither'},
    'incorrect-use-of-blockhash': {'category': 'poor-logic-flaws', 'analyzer': 'semgrep'},
    'SOLIDITY_EXACT_TIME': {'category': 'poor-logic-flaws', 'analyzer': 'smartcheck'},
    'SOLIDITY_BALANCE_EQUALITY': {'category': 'poor-logic-flaws', 'analyzer': 'smartcheck'},
    'locked-ether': {'category': 'denial-of-service', 'analyzer': 'slither'},
    'calls-loop': {'category': 'denial-of-service', 'analyzer': 'slither'},
    'msg-value-loop': {'category': 'denial-of-service', 'analyzer': 'slither'},
    'SOLIDITY_LOCKED_MONEY': {'category': 'denial-of-service', 'analyzer': 'smartcheck'},
    'SOLIDITY_TRANSFER_IN_LOOP': {'category': 'denial-of-service', 'analyzer': 'smartcheck'},
    'controlled-array-length': {'category': 'state-corruption', 'analyzer': 'slither'},
    'SOLIDITY_ARRAY_LENGTH_MANIPULATION': {'category': 'state-corruption', 'analyzer': 'smartcheck'},
    'incorrect-modifier': {'category': 'function-behavior', 'analyzer': 'slither'},
    'unchecked-transfer': {'category': 'external-call-validation', 'analyzer': 'slither'},
    'unchecked-lowlevel': {'category': 'external-call-validation', 'analyzer': 'slither'},
    'SOLIDITY_ERC20_APPROVE': {'category': 'front-running', 'analyzer': 'smartcheck'}
}

In [ ]:
available_versions = [
                "0.4.12",
                "0.4.14",
                "0.4.15",
                "0.4.16",
                "0.4.18",
                "0.4.19",
                "0.4.21",
                "0.4.23",
                "0.4.24",
                "0.4.25",
                "0.4.26",
                "0.5.0",
                "0.5.2",
                "0.5.3",
                "0.5.4",
                "0.5.5",
                "0.5.8",
                "0.5.9",
                "0.5.10",
                "0.5.11",
                "0.5.12",
                "0.5.13",
                "0.5.14",
                "0.5.15",
                "0.5.16",
                "0.5.17",
                "0.6.0",
                "0.6.2",
                "0.6.6",
                "0.6.8",
                "0.6.11",
                "0.6.12",
                "0.6.9",
                "0.7.0",
                "0.7.1",
                "0.7.3",
                "0.7.4",
                "0.7.5",
                "0.7.6",
                "0.8.1",
                "0.8.2",
                "0.8.3",
                "0.8.4",
                "0.8.5",
                "0.8.6",
                "0.8.7",
                "0.8.8",
                "0.8.9",
                "0.8.10",
                "0.8.11",
                "0.8.12",
                "0.8.13"
            ]

In [ ]:

statistics_matrices = {
    category: pd.DataFrame(
        index=pd.Index(detectors, name="detector"),
        columns=detectors,
        data=0
    )
    for category, analyzers in MAPPING.items()
    for detectors in [[detector for analyzer_detectors in analyzers.values() for detector in analyzer_detectors]]
}


In [ ]:
def get_solidity_compiler_version(solidity_code):
    # Regular expression pattern to capture only the version number part
    pattern = r"pragma\s+solidity\s+[^\d]*(\d+\.\d+\.\d+);"

    # Find all version numbers in the solidity code
    matches = re.findall(pattern, solidity_code)

    # Convert version numbers to tuples of integers for easy comparison
    version_numbers = [tuple(map(int, match.split('.'))) for match in matches]

    # If there are matches, sort them and return the oldest (smallest) version number
    if version_numbers:
        oldest_version = sorted(version_numbers)[0]  # Sorts and picks the oldest version
        return '.'.join(map(str, oldest_version))  # Converts tuple back to string format
    else:
        return None

def get_solidity_compiler_version_and_clean_code(solidity_code):
    # Regular expression pattern to capture only the version number part
    pattern_version = r"pragma\s+solidity\s+[^\d]*(\d+\.\d+\.\d+);"
    # Pattern to match entire pragma statements
    pattern_full = r"(pragma\s+solidity\s+[^\d]*\d+\.\d+\.\d+;)"

    # Find all version numbers in the solidity code
    matches = re.findall(pattern_version, solidity_code)

    # Convert version numbers to tuples of integers for easy comparison
    version_numbers = [tuple(map(int, match.split('.'))) for match in matches]

    # Initialize modified code as the original solidity code
    modified_code = solidity_code

    # If there are matches, find the oldest version
    if version_numbers:
        oldest_version = sorted(version_numbers)[-1]  # Sorts and picks the oldest version
        oldest_version_str = '.'.join(map(str, oldest_version))  # Converts tuple back to string format
        
        # Remove all pragma statements
        modified_code = re.sub(pattern_full, '', modified_code)
        
        # Add back the oldest pragma statement at the beginning
        modified_code = f"pragma solidity ^{oldest_version_str};\n" + modified_code.strip()

        return oldest_version_str, modified_code
    else:
        return None, solidity_code


def compile_solidity_contract(file_path):
    output_json = None 
    source_code = open(file_path, 'r').read()

    pragma_version = get_solidity_compiler_version(source_code)

    input_json = {
            'language': 'Solidity',
            'sources': {file_path: {'content': source_code}},
            'settings': {
                'outputSelection': {
                    '*': {
                        '*': ['metadata', 'evm.bytecode', 'evm.bytecode.sourceMap', 'abi'],
                        '': ['ast']  # Requesting AST output here
                    }
                }
            }
        }
    
    if pragma_version:
        print(f'pragma version extracted is: {pragma_version} for file: {file_path}')
        try:
            solcx.install_solc(pragma_version)
            output_json = solcx.compile_standard(input_json, allow_paths=".", solc_version=pragma_version)
        except:
            
            for pragma_version in available_versions:
                try:
                    solcx.install_solc(pragma_version)
                    #print(f'Something went wrong in compilation, trying to compile with alternative version: {pragma_version}')
                    output_json = solcx.compile_standard(input_json, allow_paths=".", solc_version=pragma_version)
                    #print(f'Compilation successful for {file_path}')
                    break
                except Exception as e:
                    #print(f'Unable to compile the contract after all attempts: {file_path}')
                    # print(e)
                    continue
    else:
        # pragma version not detected, we'll try to loop through a few available versions to see which will work
        
        for pragma_version in available_versions:
            try:
                solcx.install_solc(pragma_version)
                #print(f'Something went wrong in compilation, trying to compile with alternative version: {pragma_version}')
                output_json = solcx.compile_standard(input_json, allow_paths=".", solc_version=pragma_version)
                #print(f'Compilation successful for {file_path}')
                break
            except Exception as e:
                #print(f'Unable to compile the contract after all attempts: {file_path}')
                # print(e)
                continue
    
    return output_json

result = compile_solidity_contract(f'./{dataset_name}/QuillAudits-1inch-token/1inch-token-99fd056f91005ca521a02a005f7bcd8f77e06afc/contracts/ERC20Permit.sol')
print(result)

In [ ]:
dataset_name = 'compilable_DAppSCAN'

In [ ]:

def get_solidity_compiler_version(solidity_code):
    # Regular expression pattern to capture only the version number part
    pattern = r"pragma\s+solidity\s+[^\d]*(\d+\.\d+\.\d+);"

    # Find all version numbers in the solidity code
    matches = re.findall(pattern, solidity_code)

    # Convert version numbers to tuples of integers for easy comparison
    version_numbers = [tuple(map(int, match.split('.'))) for match in matches]

    # If there are matches, sort them and return the oldest (smallest) version number
    if version_numbers:
        oldest_version = sorted(version_numbers)[0]  # Sorts and picks the oldest version
        return '.'.join(map(str, oldest_version))  # Converts tuple back to string format
    else:
        return None

def get_solidity_compiler_version_and_clean_code(solidity_code):
    # Regular expression pattern to capture only the version number part
    pattern_version = r"pragma\s+solidity\s+[^\d]*(\d+\.\d+\.\d+);"
    # Pattern to match entire pragma statements
    pattern_full = r"(pragma\s+solidity\s+[^\d]*\d+\.\d+\.\d+;)"

    # Find all version numbers in the solidity code
    matches = re.findall(pattern_version, solidity_code)

    # Convert version numbers to tuples of integers for easy comparison
    version_numbers = [tuple(map(int, match.split('.'))) for match in matches]

    # Initialize modified code as the original solidity code
    modified_code = solidity_code

    # If there are matches, find the oldest version
    if version_numbers:
        oldest_version = sorted(version_numbers)[0]  # Sorts and picks the oldest version
        oldest_version_str = '.'.join(map(str, oldest_version))  # Converts tuple back to string format
        
        # Remove all pragma statements
        modified_code = re.sub(pattern_full, '', modified_code)
        
        # Add back the oldest pragma statement at the beginning
        modified_code = f"pragma solidity ^{oldest_version_str};\n" + modified_code.strip()

        return oldest_version_str, modified_code
    else:
        return None, solidity_code


def compile_solidity_contract(file_path):
    output_json = None 
    source_code = open(file_path, 'r').read()

    pragma_version = get_solidity_compiler_version(source_code)

    input_json = {
            'language': 'Solidity',
            'sources': {file_path: {'content': source_code}},
            'settings': {
                'outputSelection': {
                    '*': {
                        '*': ['metadata', 'evm.bytecode', 'evm.bytecode.sourceMap', 'abi'],
                        '': ['ast']  # Requesting AST output here
                    }
                }
            }
        }
    
    if pragma_version:
        print(f'pragma version extracted is: {pragma_version} for file: {file_path}')
        try:
            solcx.install_solc(pragma_version)
            output_json = solcx.compile_standard(input_json, allow_paths=".", solc_version=pragma_version)
        except:
            for pragma_version in available_versions:
                try:
                    solcx.install_solc(pragma_version)
                    print(f'Something went wrong in compilation, trying to compile with alternative version: {pragma_version}')
                    output_json = solcx.compile_standard(input_json, allow_paths=".", solc_version=pragma_version)
                    print(f'Alternative version {pragma_version} compilation successful for {file_path}')
                    break
                except Exception as e:
                    # print(f'Unable to compile the contract after all attempts: {file_path}')
                    # print(e)
                    continue
    else:
        # pragma version not detected, we'll try to loop through a few available versions to see which will work
        
        for pragma_version in available_versions:
            try:
                solcx.install_solc(pragma_version)
                print(f'Something went wrong in compilation, trying to compile with alternative version: {pragma_version}')
                output_json = solcx.compile_standard(input_json, allow_paths=".", solc_version=pragma_version)
                print(f'Alternative version {pragma_version} compilation successful for {file_path}')
                break
            except Exception as e:
                # print(f'Unable to compile the contract after all attempts: {file_path}')
                # print(e)
                continue
    
    return output_json


def print_contract_ast(compiled_sol):
    nodes = solcast.from_standard_output(compiled_sol)
    for node in nodes:
        print(node)
        # Example to explore further: print the names of contracts in a SourceUnit
        for child in node['nodes']:
            if child['nodeType'] == 'ContractDefinition':
                print(f"Contract Name: {child['name']}")


def get_enclosing_function_and_contract(output_json, solidity_filename, start_line, end_line, root_node, source_code):

    function = None
    contract = None

    
    lines = source_code.split("\n")
    start_index = sum(len(lines[i]) + 1 for i in range(start_line - 1))
    start_index_adjusted = start_index + \
        len(lines[start_line - 1]) - len(lines[start_line - 1].lstrip())
    end_index = start_index + \
        sum(len(lines[i]) + 1 for i in range(start_line - 1, end_line)) - 1

    result = root_node.children(
        required_offset=(start_index_adjusted, end_index))

    for node in result:
        if node.nodeType == 'ContractDefinition':
            contract = {
                "offset": node.offset,
                "name": node.name
            }
        elif node.nodeType == 'FunctionDefinition':
            function = {
                "offset": node.offset,
                "name": node.name
            }

    return {
        "contract": contract,
        "function": function
    }


def count_json_files_recursively(start_path='.'):
    json_file_count = 0
    for root, dirs, files in os.walk(start_path):
        for file in files:
            if file.endswith('.json'):
                json_file_count += 1
    return json_file_count


In [ ]:

def update_statistics(findings_for_file):
    global statistics_matrices
    for detector, findings in findings_for_file.items():
        for finding in findings:
            for other_detector, other_findings in findings_for_file.items():

                if other_detector == detector or REVERSE_MAPPING[detector]['category'] != REVERSE_MAPPING[other_detector]['category']:
                    continue
                for other_finding in other_findings:
                    if finding['function'] == other_finding['function']:
                        statistics_matrices[f"{REVERSE_MAPPING[detector]['category']}"][f"{detector}"][f"{other_detector}"] += 1


def process_json_file(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # Dictionary to hold the processed information
    processed_info = {}

    # Loop through each file in the JSON
    for file_name, analysis_info in data.items():
        file_details = {}
        # Loop through each detector for the current file
        for detector, detector_info in analysis_info.items():
            # Check if the analysis was successful and vulnerability findings are not empty
            if detector_info.get("successfull_analysis", False) and detector_info.get("vulnerability_findings"):
                file_details[detector] = detector_info["vulnerability_findings"]

        # Only add details for this file if there are any non-empty findings
        if file_details:
            processed_info[file_name] = file_details

    # At this point, processed_info contains the extracted data without empty findings
    # You can process this data further as needed
    print(
        f"Processed vulnerabilties.json file of the following project => {json_file_path}:")
    for file, file_information in processed_info.items():
        
        relative_file_path = f"./{dataset_name}{json_file_path[17:-21]}/{file}"
        
        output_json = compile_solidity_contract(relative_file_path)
        if output_json is None:
            # file is not compilable; just pass to the next one.
            continue

        #nodes = solcast.from_standard_output(output_json)
        root_node = solcast.from_ast(output_json["sources"][relative_file_path]["ast"])
        source_code = open(relative_file_path, 'r').read()
        
        findings_for_file = dict()
        for analyzer, analysis_results in file_information.items():
            # print(f"Analyzer is: {analyzer};")
            # print(analysis_results)
            for i in range(len(analysis_results)):
                vuln_start_line = analysis_results[i]["vulnerability_from_line"]
                vuln_end_line = analysis_results[i]["vulnerability_to_line"]
                vulnerability_name = analysis_results[i]["name"]
                # print(
                #    f"## Vulnerability is: {vulnerability_name}; vulnerability start line is: {vuln_start_line}; end line is: {vuln_end_line}")
                # print("* * * * * * * * * * * * * * * * * *")
                # print(
                #    f"start line: {vuln_start_line}, end line: {vuln_end_line}")
                if vuln_start_line == None:
                    if vuln_end_line == None:
                        break
                    else:
                        vuln_start_line = vuln_end_line
                elif vuln_end_line == None:
                    vuln_end_line = vuln_start_line

                # commenting for now, as I am doing another part of the code:
                    
                

                result = get_enclosing_function_and_contract(
                    output_json, relative_file_path, vuln_start_line, vuln_end_line, root_node, source_code)
                    
                if vulnerability_name not in REVERSE_MAPPING:
                    continue
                # else:
                #     print(
                #         f">>> The {vulnerability_name} detector name is indeed in reverse mapping <<<")
                if vulnerability_name in findings_for_file:
                    findings_for_file[f"{vulnerability_name}"].append({
                        'function': result['function']['name'] if result['function'] is not None else result['function'],
                        'contract': result['contract']['name'] if result['contract'] is not None else result['contract'],
                    })
                else:
                    findings_for_file.update({
                        f"{vulnerability_name}": [{
                            'function': result['function']['name'] if result['function'] is not None else result['function'],
                            'contract': result['contract']['name'] if result['contract'] is not None else result['contract'],
                        },]
                    })
                # print(
                #    f"Contract is: {result['contract']}, Function is: {result['function']}")
        
        if findings_for_file:
            print(f"For file {file}; findings are: ")
            print(findings_for_file)
            update_statistics(findings_for_file)
            print('# # # # # # # # # # # # # # # # # #')
    print("------------------------------------------------------------ Project finished, moving to the next one ----------------------------------------------------------- ")

    # For demonstration, just returning the processed data

    return processed_info


def read_and_process_json_files(start_directory):
    for root, dirs, files in os.walk(start_directory):
        for file in files:
            if file == 'vulnerabilities.json':
                json_file_path = os.path.join(root, file)
                # Process the JSON file
                processed_data = process_json_file(json_file_path)
                # Add your processing code here, using the 'processed_data' variable



In [ ]:
read_and_process_json_files('.')

print('\n ------------------------------ All supposed to be finished by this point ----------------------------------')


In [ ]:
%lprun -f read_and_process_json_files -f process_json_file -f get_enclosing_function_and_contract read_and_process_json_files('.')
#%lprun -f get_enclosing_function_and_contract read_and_process_json_files('.')
# %lprun -f read_and_process_json_files -f compile_solidity_contract read_and_process_json_files('.')

In [ ]:
statistics_matrices['reentrancy']

In [ ]:
statistics_matrices['low-level-calls']

In [ ]:
statistics_matrices['access-control']

In [ ]:
statistics_matrices['arithmetic']

In [ ]:
statistics_matrices['shadowing']

In [ ]:
statistics_matrices['compliance']

In [ ]:
statistics_matrices['compliance']

In [ ]:
statistics_matrices['timestamp']

In [ ]:
statistics_matrices['initialization']

In [ ]:
statistics_matrices['poor-logic-flaws']

In [ ]:
statistics_matrices['initialization']

In [ ]:
statistics_matrices['denial-of-service']

In [ ]:
statistics_matrices['state-corruption']

In [ ]:
statistics_matrices['external-call-validation']